In [6]:
# import important libraries
import pandas as pd
import sqlite3 as sql
conn = sql.connect("chin.db")
conn
def sql_query(q):
    return pd.read_sql_query(q, conn)

In [3]:
q = '''
    SELECT 
        iv.customer_id,
         CONCAT(c.first_name, ' ', c.last_name) AS full_name,
        iv.billing_country as country, 
       SUM(iv.total) AS top_buyer
    FROM 
        invoice as iv
        inner join customer as c
        on c.customer_id=iv.customer_id
    GROUP BY 
        iv.customer_id
    HAVING 
       top_buyer = (
            SELECT MAX(top_buyer_inner)
            FROM (
                SELECT  iv2.billing_country, iv2.customer_id, SUM(iv2.total) AS top_buyer_inner
                FROM  invoice as iv2
                GROUP BY 
                    iv2.customer_id, iv2.billing_country
            ) as inner_result
            WHERE inner_result.billing_country = iv.billing_country
        )
    ORDER BY 
        iv.billing_country
'''
sql_query(q)  

,customer_id,full_name,country,top_buyer
0,56,Diego Gutiérrez,Argentina,39.60
1,55,Mark Taylor,Australia,81.18
2,7,Astrid Gruber,Austria,69.30
3,8,Daan Peeters,Belgium,60.39
4,1,Luís Gonçalves,Brazil,108.90
5,3,François Tremblay,Canada,99.99
6,57,Luis Rojas,Chile,97.02
7,5,František Wichterlová,Czech Republic,144.54
8,9,Kara Nielsen,Denmark,37.62
9,44,Terhi Hämäläinen,Finland,79.20


In [142]:
q = '''

WITH RankedBuyers AS (
    SELECT 
        iv.customer_id, 
        c.country,  
        SUM(iv.subtotal) AS top_buyer,
        ROW_NUMBER() OVER (PARTITION BY c.country ORDER BY SUM(iv.subtotal) DESC) AS rank
    FROM invoice AS iv
    INNER JOIN customer AS c ON c.customer_id = iv.customer_id
    GROUP BY iv.customer_id
)


SELECT customer_id, country, top_buyer
FROM RankedBuyers
WHERE rank = 1
ORDER BY country;







'''
sql_query(q)  

,customer_id,country,top_buyer
0,56,Argentina,39.60
1,55,Australia,81.18
2,7,Austria,69.30
3,8,Belgium,60.39
4,1,Brazil,108.90
5,3,Canada,99.99
6,57,Chile,97.02
7,5,Czech Republic,144.54
8,9,Denmark,37.62
9,44,Finland,79.20


In [78]:
q = '''

SELECT 
    iv.customer_id,
    iv.billing_country,
    SUM(iv.total) AS top_buyer
FROM 
    invoice AS iv
GROUP BY 
    iv.customer_id

    '''

sql_query(q)  



,customer_id,billing_country,top_buyer
0,1,Brazil,108.90
1,2,Germany,82.17
2,3,Canada,99.99
3,4,Norway,72.27
4,5,Czech Republic,144.54
5,6,Czech Republic,128.70
6,7,Austria,69.30
7,8,Belgium,60.39
8,9,Denmark,37.62
9,10,Brazil,60.39
